In [1]:
import os
hdfs_endpoint=os.environ['HDFS_ENDPOINT']
%env HDFS_PATH /user/dominospark/small_data
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python


env: HDFS_PATH=/user/dominospark/small_data
env: HADOOP_HOME=/usr/lib/hadoop
env: HADOOP_CONF_DIR=/etc/hadoop/conf
env: HADOOP_YARN_HOME=/usr/lib/hadoop
env: HADOOP_MAPRED_HOME=/usr/lib/hadoop
env: HADOOP_HDFS_HOME=/usr/lib/hadoop-hdfs
env: SPARK_HOME=/usr/lib/spark
env: SPARK_CONF_DIR=/etc/spark/conf
env: PYSPARK_PYTHON=/opt/conda/bin/python
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python


In [3]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random
sys.path.insert(1, '/mnt/code/python/')
count=10

src_folder='/user/dominospark/large-data-temp/'
dest_folder='/user/dominospark/smalldata-10'

!hdfs dfs -rmr '/user/dominospark/smalldata-10'

import filter_data_in_hdfs
args=[count,hdfs_endpoint,src_folder,dest_folder,'local[*]']
filter_data_in_hdfs.main(args)


rmr: DEPRECATED: Please use '-rm -r' instead.
rmr: `/user/dominospark/smalldata-10': No such file or directory


In [4]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

In [5]:

sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", hdfs_endpoint) \
.getOrCreate()


In [6]:

columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])


df = sparkSession.read.csv(dest_folder, schema=columns)
df.show()
df.count()


+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 50| 57| 20|
|  1| 57| 52| 49|
|  2|  7| 50| 12|
|  3| 60| 65| 30|
|  4| 66| 31| 85|
|  5| 69| 98| 99|
|  6|  4| 28|  7|
|  7|  7| 11| 83|
|  8| 74| 95| 88|
|  9| 20| 34| 38|
+---+---+---+---+



10

In [7]:
df_load = sparkSession.read.csv(dest_folder,columns)
df_load.where(df_load.id < 5).show()


+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 50| 57| 20|
|  1| 57| 52| 49|
|  2|  7| 50| 12|
|  3| 60| 65| 30|
|  4| 66| 31| 85|
+---+---+---+---+



In [8]:
sparkSession.stop()

In [9]:
!hdfs dfs -ls '/user/dominospark/smalldata-10'

Found 2 items
-rw-r--r--   1 dominospark dominospark          0 2021-08-13 19:36 /user/dominospark/smalldata-10/_SUCCESS
-rw-r--r--   1 dominospark dominospark        106 2021-08-13 19:36 /user/dominospark/smalldata-10/part-00000-ae995e96-dbc3-499f-b699-2507813dc9b7-c000.csv
